# Strands Agents on the Edge

> **AWS re:Invent 2025 Session DEV301** | *AI agents at the edge: Build for offline, scale in cloud*

This notebook demonstrates building Generative AI agents that operate fully offline using the **Strands Agents SDK**. Learn how to deploy lightweight local models with Ollama and integrate custom tools for remote manufacturing, field operations, and data sovereignty scenarios.

**Key Concepts:**
- Edge-deployed agents that work without internet connectivity
- Seamless scaling to cloud capabilities (e.g., Claude on Amazon Bedrock) when connectivity is available
- Practical patterns for combining local processing with cloud AI

## Demo Overview

1. **Tool Decorator** - Create custom tools using the `@tool` decorator for IoT device control
2. **Structured Output** - Extract type-safe, validated data from LLM responses using Pydantic models for SCADA/MES integration
3. **Session Management** - Persist agent conversations across interactions using FileSessionManager for edge deployments with intermittent connectivity
4. **MCP Integration** - Connect to local SQLite databases via Model Context Protocol for offline operation

Each demo is self-contained and demonstrates real-world edge computing scenarios where AI agents can operate with limited connectivity and local resources.

---

**Speakers:** Ana Cunha, David Victoria  
**Resources:** [Strands Agents SDK](https://github.com/strands-agents/sdk-python) | [Ollama](https://ollama.ai)

## Setup & Dependencies

First, let's install the required packages. Run the cell below to install all dependencies.

In [ ]:
# Install required dependencies
%pip install 'strands-agents[ollama]' strands-agents-tools pydantic mcp hypothesis

# Note: For Demo 4 (MCP Integration), you'll also need 'uvx' installed.
# uvx is part of the 'uv' Python package manager.
# Install it with: pip install uv
# Or follow instructions at: https://docs.astral.sh/uv/getting-started/installation/

# Note: Ensure Ollama is running locally with the model pulled:
# ollama pull hoangquan456/qwen3-nothink:4b
# ollama serve

In [ ]:
# Configure the Ollama model for all demos
# This uses a local Ollama instance running the qwen3-nothink model
from strands.models.ollama import OllamaModel

OLLAMA_MODEL_ID = 'hoangquan456/qwen3-nothink:4b'

# Create a shared Ollama model instance for edge deployment
# Using a lightweight model suitable for edge devices
OLLAMA_MODEL = OllamaModel(
    host="http://localhost:11434",  # Local Ollama server
    model_id=OLLAMA_MODEL_ID,  # Edge-optimized model
    temperature=0.7,
    keep_alive="10m"  # Keep model loaded for faster subsequent calls
)

print("Ollama model configured!")
print(f"  Host: http://localhost:11434")
print(f"  Model: {OLLAMA_MODEL_ID}")

---

## Demo 1: Tool Decorator - IoT Device Control

The `@tool` decorator is a powerful feature that transforms ordinary Python functions into agent-callable tools. This is particularly valuable for **edge IoT scenarios** where agents need to interact with local sensors, actuators, and industrial equipment.

### Why Tool Decorator for Edge?

- **Local Device Integration**: Connect agents directly to sensors, PLCs, and actuators
- **Custom Protocols**: Wrap proprietary industrial protocols (Modbus, OPC-UA) as simple tools
- **Offline Operation**: Tools run locally without cloud dependencies
- **Type Safety**: Docstrings and type hints automatically generate tool specifications for the LLM

In this demo, we'll create tools to read sensor data and control actuators on a simulated IoT device network.

In [ ]:
import random
from strands import Agent, tool

# Simulated IoT device registry - in production, this would connect to real hardware
DEVICES = {
    "temp-sensor-001": {
        "type": "temperature",
        "location": "production_floor",
        "unit": "celsius",
        "min_value": 15.0,
        "max_value": 35.0
    },
    "humidity-sensor-001": {
        "type": "humidity",
        "location": "storage_area",
        "unit": "percent",
        "min_value": 30.0,
        "max_value": 70.0
    },
    "valve-actuator-001": {
        "type": "valve",
        "location": "cooling_system",
        "states": ["open", "closed", "partial"],
        "current_state": "closed"
    }
}


@tool
def read_sensor(device_id: str) -> str:
    """
    Read current sensor values from an IoT device.
    
    Args:
        device_id: The unique identifier of the sensor device (e.g., 'temp-sensor-001')
    
    Returns:
        A string containing the sensor reading with value and unit, or an error message
    """
    if device_id not in DEVICES:
        return f"Error: Device '{device_id}' not found. Available devices: {list(DEVICES.keys())}"
    
    device = DEVICES[device_id]
    
    # Check if this is a sensor (has min/max values)
    if "min_value" not in device:
        return f"Error: Device '{device_id}' is not a sensor (type: {device['type']})"
    
    # Simulate sensor reading within device range
    value = round(random.uniform(device["min_value"], device["max_value"]), 2)
    
    return f"Device: {device_id}\nType: {device['type']}\nLocation: {device['location']}\nReading: {value} {device['unit']}"


@tool
def control_device(device_id: str, action: str) -> str:
    """
    Send a control command to an IoT actuator device.
    
    Args:
        device_id: The unique identifier of the actuator device (e.g., 'valve-actuator-001')
        action: The control action to perform (e.g., 'open', 'closed', 'partial')
    
    Returns:
        A string confirming the action taken or an error message
    """
    if device_id not in DEVICES:
        return f"Error: Device '{device_id}' not found. Available devices: {list(DEVICES.keys())}"
    
    device = DEVICES[device_id]
    
    # Check if this is an actuator (has states)
    if "states" not in device:
        return f"Error: Device '{device_id}' is not controllable (type: {device['type']})"
    
    # Validate the action
    if action not in device["states"]:
        return f"Error: Invalid action '{action}'. Valid actions for {device_id}: {device['states']}"
    
    # Update device state
    previous_state = device["current_state"]
    device["current_state"] = action
    
    return f"Device: {device_id}\nType: {device['type']}\nLocation: {device['location']}\nAction: {previous_state} -> {action}\nStatus: SUCCESS"


print("IoT tools defined successfully!")
print(f"\nAvailable devices: {list(DEVICES.keys())}")

### Using the Tools with an Agent

Now let's create an agent equipped with our IoT tools. The agent can understand natural language requests and automatically invoke the appropriate tools to interact with our simulated devices.

Notice how the `@tool` decorator uses the function's:
- **Docstring** → Tool description for the LLM
- **Type hints** → Parameter types and return type
- **Argument descriptions** → Parameter documentation

In [ ]:
# Create an agent with our IoT tools using the local Ollama model
iot_agent = Agent(
    model=OLLAMA_MODEL,
    tools=[read_sensor, control_device],
    system_prompt="You are an IoT control agent for an industrial facility. Help users monitor sensors and control actuators. Always report the exact values returned by tools."
)

# Demo: Read sensor values
print("=" * 60)
print("DEMO: Reading sensor values")
print("=" * 60)
response = iot_agent("What is the current temperature on the production floor? Use the temp-sensor-001 device.")
print(f"\nAgent Response:\n{response}")

In [ ]:
# Demo: Control an actuator
print("=" * 60)
print("DEMO: Controlling an actuator")
print("=" * 60)
response = iot_agent("Open the cooling system valve (valve-actuator-001) to help reduce the temperature.")
print(f"\nAgent Response:\n{response}")

In [ ]:
# Demo: Multi-step operation - read humidity and take action
print("=" * 60)
print("DEMO: Multi-step operation")
print("=" * 60)
response = iot_agent("Check the humidity in the storage area using humidity-sensor-001, and tell me if it's within acceptable range (40-60%).")
print(f"\nAgent Response:\n{response}")

### Key Takeaways - Tool Decorator

✅ The `@tool` decorator transforms Python functions into agent-callable tools  
✅ Type hints and docstrings automatically generate tool specifications  
✅ Tools can wrap any local functionality - sensors, databases, APIs  
✅ Perfect for edge deployments where agents need direct hardware access  

---

## Demo 2: Structured Output - SCADA/MES Integration

**Structured Output** allows agents to return type-safe, validated data using Pydantic models. This is essential for **industrial edge scenarios** where data must conform to strict schemas for SCADA (Supervisory Control and Data Acquisition) and MES (Manufacturing Execution System) integration.

### Why Structured Output for Edge?

- **Type Safety**: Guarantee data conforms to expected schemas before processing
- **Validation**: Pydantic automatically validates field types, ranges, and constraints
- **Integration Ready**: Output directly maps to industrial data formats (OPC-UA, ISA-95)
- **Nested Models**: Complex hierarchical data structures for equipment, sensors, and alarms
- **Error Prevention**: Catch malformed data at the edge before it propagates to control systems

In this demo, we'll extract structured production metrics from a simulated SCADA system response.

In [ ]:
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel, Field
from strands import Agent

# Define Pydantic models for industrial data extraction

class SensorReading(BaseModel):
    """Represents a single sensor measurement from industrial equipment."""
    sensor_id: str = Field(description="Unique identifier for the sensor")
    value: float = Field(description="The measured value")
    unit: str = Field(description="Unit of measurement (e.g., celsius, percent, m/s)")
    timestamp: Optional[str] = Field(default=None, description="ISO format timestamp of the reading")


class AlarmInfo(BaseModel):
    """Represents an active alarm or warning on industrial equipment."""
    alarm_id: str = Field(description="Unique identifier for the alarm")
    severity: str = Field(description="Alarm severity: low, medium, high, or critical")
    message: str = Field(description="Human-readable alarm description")
    acknowledged: bool = Field(default=False, description="Whether the alarm has been acknowledged")


class EquipmentStatus(BaseModel):
    """Represents the status of a piece of industrial equipment."""
    equipment_id: str = Field(description="Unique identifier for the equipment")
    name: str = Field(description="Human-readable equipment name")
    status: str = Field(description="Current status: running, stopped, maintenance, or fault")
    readings: List[SensorReading] = Field(default_factory=list, description="Current sensor readings")
    active_alarms: List[AlarmInfo] = Field(default_factory=list, description="Active alarms on this equipment")


class ProductionMetrics(BaseModel):
    """Represents production metrics for a manufacturing line."""
    line_id: str = Field(description="Production line identifier")
    shift: str = Field(description="Current shift (e.g., morning, afternoon, night)")
    units_produced: int = Field(description="Number of units produced this shift")
    units_target: int = Field(description="Target units for this shift")
    efficiency_percent: float = Field(description="Production efficiency as a percentage")
    equipment: List[EquipmentStatus] = Field(default_factory=list, description="Status of equipment on this line")


print("Pydantic models defined successfully!")
print("\nModel hierarchy:")
print("  ProductionMetrics")
print("    └── EquipmentStatus (list)")
print("          ├── SensorReading (list)")
print("          └── AlarmInfo (list)")

### Extracting Structured Data from SCADA Response

Now let's use the agent's `structured_output()` method to extract typed data from a simulated SCADA system response. The agent will parse the unstructured text and return a validated `ProductionMetrics` object.

In [ ]:
# Simulated SCADA response - in production, this would come from an OPC-UA server or industrial API
SCADA_RESPONSE = """
Production Line A Status Report
================================
Line ID: LINE-A
Current Shift: Morning

Equipment Status:
-----------------
1. Conveyor Belt CB-101
   - Equipment ID: CB-101
   - Status: Running
   - Sensors:
     * Motor Temperature (TEMP-CB101-M1): 45.2°C
     * Belt Speed (SPEED-CB101): 2.5 m/s
   - Alarms:
     * ALM-001: High temperature warning (Severity: medium) - Not acknowledged

2. Packaging Unit PU-201
   - Equipment ID: PU-201
   - Status: Running
   - Sensors:
     * Cycle Time (CYCLE-PU201): 3.2 seconds
     * Units Packed Counter (COUNT-PU201): 1247 units
   - Alarms: None active

Production Summary:
-------------------
Units Produced: 1,247
Shift Target: 1,500
Current Efficiency: 83.1%
"""

print("Simulated SCADA Response:")
print(SCADA_RESPONSE)

In [ ]:
# Create an agent for structured data extraction using the local Ollama model
extraction_agent = Agent(
    model=OLLAMA_MODEL,
    system_prompt="You are a data extraction agent for industrial systems. Extract structured data from SCADA reports accurately."
)

# Use structured_output_model parameter to extract typed data (new API)
print("=" * 60)
print("DEMO: Extracting structured production metrics")
print("=" * 60)

result = extraction_agent(
    f"Extract the production metrics from this SCADA report:\n\n{SCADA_RESPONSE}",
    structured_output_model=ProductionMetrics
)
metrics = result.structured_output

print(f"\nExtraction successful! Type: {type(metrics).__name__}")

In [ ]:
# Demonstrate accessing individual fields from the structured output
print("=" * 60)
print("DEMO: Accessing structured data fields")
print("=" * 60)

# Top-level production metrics
print(f"\n📊 Production Line: {metrics.line_id}")
print(f"   Shift: {metrics.shift}")
print(f"   Units Produced: {metrics.units_produced} / {metrics.units_target}")
print(f"   Efficiency: {metrics.efficiency_percent}%")

# Equipment status with nested data
print(f"\n🏭 Equipment Status ({len(metrics.equipment)} units):")
for equip in metrics.equipment:
    print(f"\n   [{equip.equipment_id}] {equip.name}")
    print(f"   Status: {equip.status}")
    
    # Sensor readings
    if equip.readings:
        print(f"   Sensors:")
        for reading in equip.readings:
            print(f"     - {reading.sensor_id}: {reading.value} {reading.unit}")
    
    # Active alarms
    if equip.active_alarms:
        print(f"   ⚠️ Active Alarms:")
        for alarm in equip.active_alarms:
            ack_status = "✓" if alarm.acknowledged else "✗"
            print(f"     - [{alarm.severity.upper()}] {alarm.message} (Ack: {ack_status})")
    else:
        print(f"   ✅ No active alarms")

### Key Takeaways - Structured Output

✅ `structured_output()` extracts validated, typed data from unstructured text  
✅ Pydantic models define the exact schema for industrial data  
✅ Nested models handle complex hierarchies (equipment → sensors → alarms)  
✅ Type-safe field access enables reliable downstream processing  
✅ Perfect for SCADA/MES integration where data integrity is critical  

---

## Demo 3: Session Management - Persistent Edge Conversations

**Session Management** enables agents to persist their state and conversation history across interactions. This is critical for **edge deployments** where devices may have intermittent connectivity, power cycles, or need to maintain context across restarts.

### Why Session Management for Edge?

- **Intermittent Connectivity**: Edge devices often lose network connection; sessions preserve state locally until sync
- **Power Resilience**: Survive unexpected shutdowns and resume conversations after restart
- **Context Continuity**: Maintain conversation history for ongoing diagnostic or monitoring tasks
- **Local-First Storage**: FileSessionManager stores sessions on local filesystem - no cloud dependency
- **Multi-Session Support**: Track multiple concurrent conversations (e.g., different operators or equipment)

In this demo, we'll create an agent with persistent sessions that survives across multiple interactions and can be restored later.

In [ ]:
import os
import shutil
from strands import Agent
from strands.session.file_session_manager import FileSessionManager

# Define a local storage directory for edge sessions
SESSIONS_DIR = "./edge_sessions"

# Clean up any previous demo sessions for a fresh start
if os.path.exists(SESSIONS_DIR):
    shutil.rmtree(SESSIONS_DIR)
os.makedirs(SESSIONS_DIR, exist_ok=True)

# Create a FileSessionManager with a unique session ID
# This simulates an edge device maintaining state for a specific operator or task
SESSION_ID = "edge-operator-001"

session_manager = FileSessionManager(
    session_id=SESSION_ID,
    storage_dir=SESSIONS_DIR
)

print(f"Session Manager created!")
print(f"  Session ID: {SESSION_ID}")
print(f"  Storage Directory: {SESSIONS_DIR}")

### Building Conversation History

Now let's create an agent with the session manager and have a multi-turn conversation. Each message is automatically persisted to the local filesystem.

In [ ]:
# Create an agent with session management for edge diagnostics using the local Ollama model
edge_agent = Agent(
    model=OLLAMA_MODEL,
    session_manager=session_manager,
    system_prompt="You are an edge device diagnostic assistant. Help operators troubleshoot equipment issues. Keep responses concise and technical."
)

print("=" * 60)
print("DEMO: Building conversation history with session persistence")
print("=" * 60)

# First interaction - report an issue
print("\n[Message 1] Operator reports an issue...")
response1 = edge_agent("The conveyor belt CB-101 is showing high motor temperature. What should I check first?")
print(f"Agent: {response1}")

# Second interaction - follow-up question
print("\n[Message 2] Operator follows up...")
response2 = edge_agent("I checked the motor and it looks fine. The belt tension seems loose though.")
print(f"Agent: {response2}")

# Third interaction - request summary
print("\n[Message 3] Operator requests summary...")
response3 = edge_agent("Can you summarize what we've discussed so far about CB-101?")
print(f"Agent: {response3}")

In [ ]:
# Display session state and message count
print("=" * 60)
print("DEMO: Examining session state")
print("=" * 60)

# Access the agent's message history
message_count = len(edge_agent.messages)
print(f"\n📊 Session Statistics:")
print(f"   Session ID: {SESSION_ID}")
print(f"   Total Messages: {message_count}")

# Show the session files created on disk
print(f"\n📁 Session Files on Disk:")
for root, dirs, files in os.walk(SESSIONS_DIR):
    level = root.replace(SESSIONS_DIR, '').count(os.sep)
    indent = '   ' * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = '   ' * (level + 1)
    for file in files:
        print(f"{subindent}{file}")

### Session Restoration - Simulating Device Restart

Now let's simulate what happens when an edge device restarts or loses power. We'll create a **new agent instance** with the same session ID, and it will automatically restore the conversation history.

In [ ]:
# Simulate device restart by creating a completely new agent with the same session ID
print("=" * 60)
print("DEMO: Simulating device restart - restoring session")
print("=" * 60)

print("\n🔄 Simulating edge device restart...")
print("   (Creating new agent instance with same session ID)")

# Create a NEW session manager with the SAME session ID
restored_session_manager = FileSessionManager(
    session_id=SESSION_ID,  # Same session ID as before
    storage_dir=SESSIONS_DIR
)

# Create a NEW agent - it will automatically restore the session
restored_agent = Agent(
    model=OLLAMA_MODEL,
    session_manager=restored_session_manager,
    system_prompt="You are an edge device diagnostic assistant. Help operators troubleshoot equipment issues. Keep responses concise and technical."
)

# Verify the conversation history was restored
restored_message_count = len(restored_agent.messages)
print(f"\n✅ Session restored successfully!")
print(f"   Messages recovered: {restored_message_count}")

In [ ]:
# Continue the conversation from where we left off
print("=" * 60)
print("DEMO: Continuing conversation after restart")
print("=" * 60)

# The agent remembers the previous context about CB-101
print("\n[Message 4] Operator continues after device restart...")
response4 = restored_agent("I adjusted the belt tension on CB-101. What should I monitor to confirm the fix worked?")
print(f"Agent: {response4}")

# Verify the agent has full context
print("\n[Message 5] Verify context retention...")
response5 = restored_agent("What was the original problem I reported?")
print(f"Agent: {response5}")

# Final message count
final_message_count = len(restored_agent.messages)
print(f"\n📊 Final Session Statistics:")
print(f"   Total Messages (after restoration + new): {final_message_count}")

### Key Takeaways - Session Management

✅ `FileSessionManager` persists agent state and conversation history to local filesystem  
✅ Sessions survive application restarts, power cycles, and connectivity loss  
✅ Creating a new agent with the same session ID automatically restores context  
✅ Perfect for edge deployments with intermittent connectivity  
✅ No cloud dependency - all data stored locally on the edge device  

---

## Demo 4: MCP Integration - Local SQLite Database

The **Model Context Protocol (MCP)** is an open protocol that standardizes how applications provide context and tools to Large Language Models. For **edge deployments**, MCP enables agents to connect to local databases and services without cloud dependencies.

### Why MCP for Edge?

- **Offline Operation**: Connect to local SQLite databases that work without internet connectivity
- **Standardized Protocol**: Use the same interface for local and remote data sources
- **Tool Discovery**: MCP servers expose their capabilities as discoverable tools
- **Data Persistence**: Store and query edge device data, logs, and configurations locally
- **Ecosystem Integration**: Leverage existing MCP servers for databases, filesystems, and more

In this demo, we'll connect to a local SQLite database via MCP to store and query edge device telemetry data - perfect for scenarios where edge devices need to operate autonomously and sync data when connectivity is restored.

In [ ]:
import os
from mcp import stdio_client, StdioServerParameters
from strands import Agent
from strands.tools.mcp import MCPClient

# Define the local SQLite database path for edge data storage
DB_PATH = "./edge_telemetry2.db"

# Clean up any previous demo database for a fresh start
if os.path.exists(DB_PATH):
    os.remove(DB_PATH)

# Create MCPClient with stdio transport for the SQLite MCP server
# This uses uvx to run the mcp-server-sqlite package
mcp_client = MCPClient(lambda: stdio_client(
    StdioServerParameters(
        command="uvx",
        args=["mcp-server-sqlite", "--db-path", DB_PATH]
    )
))

print(f"MCP Client configured for SQLite database: {DB_PATH}")
print("\nNote: This requires 'uvx' to be installed (pip install uv)")

In [ ]:
# Use context manager for proper MCP lifecycle management
# This ensures the connection is properly established and cleaned up
print("=" * 60)
print("DEMO: Connecting to MCP server and listing available tools")
print("=" * 60)

with mcp_client:
    # List available tools from the MCP server
    tools = mcp_client.list_tools_sync()
    
    print(f"\n✅ Connected to SQLite MCP server!")
    print(f"\n📋 Available MCP Tools ({len(tools)}):")
    for tool in tools:
        # Get tool name - try different attribute names
        tool_name = getattr(tool, 'tool_name', None) or getattr(tool, 'name', 'Unknown')
        print(f"   • {tool_name}")

### Database Operations Through MCP

Now let's create an agent that uses the MCP tools to interact with our local SQLite database. We'll create a table for edge device telemetry, insert sample data, and query it back - all through natural language commands.

In [ ]:
# Create an agent with MCP tools for database operations
print("=" * 60)
print("DEMO: Creating database schema for edge telemetry")
print("=" * 60)

with mcp_client:
    tools = mcp_client.list_tools_sync()
    
    # Create agent with MCP database tools using the local Ollama model
    db_agent = Agent(
        model=OLLAMA_MODEL,
        tools=tools,
        system_prompt="You are a database assistant for edge device telemetry. Execute SQL commands precisely as requested. Always confirm the results of operations."
    )
    
    # Create the telemetry table
    response = db_agent("""
    Create a table called 'device_telemetry' with the following columns:
    - id (INTEGER PRIMARY KEY AUTOINCREMENT)
    - device_id (TEXT NOT NULL)
    - metric_type (TEXT NOT NULL) 
    - value (REAL NOT NULL)
    - unit (TEXT NOT NULL)
    - timestamp (TEXT DEFAULT CURRENT_TIMESTAMP)
    
    Then confirm the table was created by describing its schema.
    """)
    print(f"\nAgent Response:\n{response}")

In [ ]:
# Insert sample edge device telemetry data
print("=" * 60)
print("DEMO: Inserting edge device telemetry data")
print("=" * 60)

with mcp_client:
    tools = mcp_client.list_tools_sync()
    
    db_agent = Agent(
        model=OLLAMA_MODEL,
        tools=tools,
        system_prompt="You are a database assistant for edge device telemetry. Execute SQL commands precisely as requested."
    )
    
    # Insert telemetry records
    response = db_agent("""
    Insert the following telemetry records into device_telemetry:
    
    1. device_id: 'temp-sensor-001', metric_type: 'temperature', value: 23.5, unit: 'celsius'
    2. device_id: 'temp-sensor-001', metric_type: 'temperature', value: 24.1, unit: 'celsius'
    3. device_id: 'humidity-sensor-001', metric_type: 'humidity', value: 45.2, unit: 'percent'
    4. device_id: 'humidity-sensor-001', metric_type: 'humidity', value: 47.8, unit: 'percent'
    5. device_id: 'valve-actuator-001', metric_type: 'position', value: 75.0, unit: 'percent_open'
    
    Confirm how many records were inserted.
    """)
    print(f"\nAgent Response:\n{response}")

In [ ]:
# Query the data back from the database
print("=" * 60)
print("DEMO: Querying edge telemetry data")
print("=" * 60)

with mcp_client:
    tools = mcp_client.list_tools_sync()
    
    db_agent = Agent(
        model=OLLAMA_MODEL,
        tools=tools,
        system_prompt="You are a database assistant for edge device telemetry. Execute SQL queries and present results clearly."
    )
    
    # Query all telemetry data
    response = db_agent("Show me all the telemetry records in the device_telemetry table, ordered by device_id and timestamp.")
    print(f"\nAgent Response:\n{response}")

In [ ]:
# Demonstrate analytical query
print("=" * 60)
print("DEMO: Analytical query on edge data")
print("=" * 60)

with mcp_client:
    tools = mcp_client.list_tools_sync()
    
    db_agent = Agent(
        model=OLLAMA_MODEL,
        tools=tools,
        system_prompt="You are a database assistant for edge device telemetry. Execute SQL queries and present results clearly."
    )
    
    # Run analytical query
    response = db_agent("""
    Calculate the average value for each device_id in the device_telemetry table.
    Show the device_id, metric_type, average value, and unit.
    """)
    print(f"\nAgent Response:\n{response}")

### Key Takeaways - MCP Integration

✅ MCP provides a standardized protocol for connecting agents to external tools and services  
✅ `MCPClient` with stdio transport connects to local MCP servers like SQLite  
✅ Context managers (`with` statement) ensure proper connection lifecycle management  
✅ MCP tools are discoverable - agents can list and use available capabilities  
✅ Perfect for edge deployments requiring offline database access and local data persistence  

---

## Summary: Strands Agents on the Edge

This notebook demonstrated four key capabilities of the **Strands Agents SDK** that make it ideal for building AI agents on edge devices. Here's a recap of what we covered:

### 🔧 Tool Decorator - Device Integration

The `@tool` decorator transforms Python functions into agent-callable tools, enabling direct integration with IoT sensors, actuators, and industrial equipment. Key benefits for edge:
- **Local execution** - Tools run on the edge device without cloud round-trips
- **Custom protocols** - Wrap proprietary industrial protocols (Modbus, OPC-UA) as simple functions
- **Auto-documentation** - Type hints and docstrings generate tool specifications automatically

### 📊 Structured Output - Industrial Data Extraction

The `structured_output()` method extracts validated, type-safe data using Pydantic models - essential for SCADA/MES integration where data integrity is critical. Key benefits for edge:
- **Schema enforcement** - Guarantee data conforms to industrial standards before processing
- **Nested models** - Handle complex hierarchies (equipment → sensors → alarms)
- **Integration ready** - Output maps directly to industrial data formats

### 💾 Session Management - Persistent Conversations

`FileSessionManager` persists agent state and conversation history to the local filesystem, enabling context continuity across restarts and connectivity loss. Key benefits for edge:
- **Power resilience** - Survive unexpected shutdowns and resume conversations
- **Offline-first** - All data stored locally with no cloud dependency
- **Context continuity** - Maintain diagnostic history across operator shifts

### 🔌 MCP Integration - Offline Database Access

The Model Context Protocol (MCP) provides a standardized way to connect agents to local databases and services. Key benefits for edge:
- **Offline operation** - SQLite databases work without internet connectivity
- **Tool discovery** - MCP servers expose capabilities as discoverable tools
- **Data persistence** - Store telemetry, logs, and configurations locally

---

### Next Steps

To build your own edge AI agents with Strands:

1. **Explore the SDK**: Visit [github.com/strands-agents/sdk-python](https://github.com/strands-agents/sdk-python) for documentation and examples
2. **Create custom tools**: Use `@tool` to wrap your device-specific APIs and protocols
3. **Define data models**: Build Pydantic schemas for your industrial data formats
4. **Enable persistence**: Configure `FileSessionManager` for your edge deployment
5. **Integrate MCP servers**: Connect to local databases, filesystems, and services

Happy building! 🚀